# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san luis,-33.2950,-66.3356,8.47,40,0,1.02,AR,1748465566
1,1,port-vila,-17.7338,168.3219,21.78,88,20,3.09,VU,1748465567
2,2,zhytkavichy,52.2168,27.8561,13.09,98,59,2.83,BY,1748465568
3,3,adamstown,-25.0660,-130.1015,21.56,57,65,7.60,PN,1748465569
4,4,nar'yan-mar,67.6713,53.0870,10.03,75,21,3.02,RU,1748465570


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    scale = 0.5,
    alpha=0.7,
    frame_width=800

)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[city_data_df["Max Temp"] <= 30]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san luis,-33.2950,-66.3356,8.47,40,0,1.02,AR,1748465566
1,1,port-vila,-17.7338,168.3219,21.78,88,20,3.09,VU,1748465567
2,2,zhytkavichy,52.2168,27.8561,13.09,98,59,2.83,BY,1748465568
3,3,adamstown,-25.0660,-130.1015,21.56,57,65,7.60,PN,1748465569
4,4,nar'yan-mar,67.6713,53.0870,10.03,75,21,3.02,RU,1748465570
5,5,edinburgh of the seven seas,-37.0676,-12.3116,14.10,73,77,8.71,SH,1748465571
6,6,bochum,51.4833,7.2167,12.27,82,5,2.10,DE,1748465357
7,7,puerto natales,-51.7236,-72.4875,5.75,89,99,3.26,CL,1748465573
8,8,kudahuvadhoo,2.6708,72.8944,28.64,75,100,8.53,MV,1748465574
9,9,yujing,23.1249,120.4614,24.29,95,92,1.81,TW,1748465575


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", value="")
# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,san luis,AR,-33.2950,-66.3356,40,
1,port-vila,VU,-17.7338,168.3219,88,
2,zhytkavichy,BY,52.2168,27.8561,98,
3,adamstown,PN,-25.0660,-130.1015,57,
4,nar'yan-mar,RU,67.6713,53.0870,75,
5,edinburgh of the seven seas,SH,-37.0676,-12.3116,73,
6,bochum,DE,51.4833,7.2167,82,
7,puerto natales,CL,-51.7236,-72.4875,89,
8,kudahuvadhoo,MV,2.6708,72.8944,75,
9,yujing,TW,23.1249,120.4614,95,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [49]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san luis - nearest hotel: Hotel Dos Venados
port-vila - nearest hotel: Holiday Inn Resort Vanuatu
zhytkavichy - nearest hotel: Чвэрць стагоддзя
adamstown - nearest hotel: No hotel found
nar'yan-mar - nearest hotel: Гостиница "Агат"
edinburgh of the seven seas - nearest hotel: No hotel found
bochum - nearest hotel: acora Hotel
puerto natales - nearest hotel: Dorotea Patagonia Hostel
kudahuvadhoo - nearest hotel: Hotel Riu Palace Maldivas
yujing - nearest hotel: 蘭花會館
namibe - nearest hotel: Nelsal Pensao
greymouth - nearest hotel: Recreation Hotel
tolanaro - nearest hotel: Hôtel Mahavokey
tiksi - nearest hotel: Арктика
bilibino - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
hasaki - nearest hotel: 大新旅館
sinop - nearest hotel: No hotel found
kondopoga - nearest hotel: Карелия
osakarovka - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
atafu village - nearest hotel: No hotel found
bethel - nearest hotel: Hampton In

,City,Country,Lat,Lng,Humidity,Hotel Name
0,san luis,AR,-33.2950,-66.3356,40,Hotel Dos Venados
1,port-vila,VU,-17.7338,168.3219,88,Holiday Inn Resort Vanuatu
2,zhytkavichy,BY,52.2168,27.8561,98,Чвэрць стагоддзя
3,adamstown,PN,-25.0660,-130.1015,57,No hotel found
4,nar'yan-mar,RU,67.6713,53.0870,75,"Гостиница ""Агат"""
...,...,...,...,...,...,...
574,leua,SK,48.2156,18.6071,83,Levi Dom
575,ghormach,AF,35.7306,63.7826,23,No hotel found
576,bandon,IE,51.7469,-8.7425,94,The Munster Arms Hotel
578,tuy hoa,VN,13.0833,109.3000,98,Nhu Y Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [58]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", "Lat", 
    geo=True,
    tiles="OSM",
    size=hotel_df["Humidity"] / 10,
    color="blue",
    hover_cols=["City", "Country", "Hotel Name"]
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_size,City,Country,Hotel Name)